In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gmaps
import requests

from citipy import citipy
from config import weather_api_key, g_key

gmaps.configure(api_key=g_key)

from datetime import datetime
import time

from scipy.stats import linregress

In [2]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=d4cb209770713b56aa45c40025b8cc28


In [3]:
#create random longitude and latitude combos
lats = np.random.uniform(low=-90.0, high=90.0, size = 100)
lans = np.random.uniform(low=-180.0, high=180.0,size = 100)
lat_lans = zip(lats,lans)
lat_lans

In [4]:
#list of cities
cities = []
for coordinate in lat_lans:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    if city not in cities:
        cities.append(city)
        
len(cities)

77

In [14]:
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1


    
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    #print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        weather_description = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": weather_description,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except KeyError:
        #print("City not found. Skipping...")
        pass
col = ['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Current Descriptin', 'Country', 'Date']

j = 0
cName = []
cLat  = []
cLng  = []
cMaxT = []
cHum  = []
cCld  = []
cWinS = []
cCur  = []
cCoun = []
cDate = []
#print(city_data)
for j in range(len(city_data)):
    cName += [city_data[j].get("City")]
    cLat  += [city_data[j].get("Lat")]
    cLng  += [city_data[j].get("Lng")]
    cMaxT += [city_data[j].get("Max Temp")]
    cHum  += [city_data[j].get("Humidity")]
    cCld  += [city_data[j].get("Cloudiness")]
    cWinS += [city_data[j].get("Wind Speed")]
    cCur  += [city_data[j].get("Current Description")]
    cCoun += [city_data[j].get("Country")]
    cDate += [city_data[j].get("Date")]
    
wList = [cName, cLat, cLng, cMaxT, cHum, cCld, cWinS, cCur, cCoun, cDate]
#print(wList)

    
cityDF = pd.DataFrame(wList).transpose()
cityDF.columns= col
# Indicate that Data Loading is complete.
print(cityDF)

cityDF.to_csv('weather_data.csv', index = False)
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
            City    Lat     Lng Max Temp Humidity Cloudiness Wind Speed  \
0        Bubaque  11.28  -15.83    80.42       56          5        6.6   
1      Cape Town -33.93   18.42       61       93          0       5.82   
2       Cherskiy  68.75   161.3   -19.41       87        100       5.77   
3        Rikitea -23.12 -134.97     76.1       68         42       4.14   
4        Kavieng  -2.57   150.8    85.96       69        100      10.45   
..           ...    ...     ...      ...      ...        ...        ...   
64         Pevek   69.7  170.31    14.65       89        100      18.75   
65  Mochalishche  56.33   48.35      1.4       84          0       6.71   
66        Talaya  61.38  152.75   -26.91       82        100       4.72   
67       Urengoy  65.97   78.37     5.49       95        100       8.03   
68       Qaanaaq  77.48  -69.36    23.05       90        100       4.65   

   Current Descriptin Country          